In [1]:
#@markdown #Please run this cell to install dependencies.

#@markdown In this demo notebook you can generate MIDI transcription with pitch bends from violin recordings on YouTube.



!rm -rf MUSC_violin # remove if the directory already exists
!git clone https://github.com/MTG/violin-transcription.git MUSC_violin &> /dev/null
import os
import sys
!pip install yt-dlp &> /dev/null
!pip install mir_eval &> /dev/null
!pip install pretty_midi &> /dev/null
!pip install unidecode &> /dev/null
from unidecode import unidecode
#!pip install yt-dlp &> /dev/null
import torch
import json
#from yt_dlp import YoutubeDL
sys.path.append('MUSC_violin')
from MUSC_violin import musc


In [2]:
from musc.model import PretrainedModel

device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = PretrainedModel(instrument='violin').to(device)

hop in samples: 256
main stream window: 131964 , attendant stream window: 136015 , conformer input dim: 256
main frame overlap: [574, 574] , attendant frame overlap: [2599, 2600]
crop for main pathway [2025, -2026]
Total sequence duration is 136015 samples
Main stream receptive field for one frame is 1148 samples
Attendant stream receptive field for one frame is 5199 samples
Total parameter count:  54580722


Downloading...
From: https://drive.google.com/uc?export=download&confirm=pbef&id=1FfUjC3usmZBoTxNT6rNVIcYw4pol4K1g
To: /content/MUSC_violin/musc/violin_model.pt
100%|██████████| 219M/219M [00:12<00:00, 17.0MB/s]


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


An example outside the train set

In [4]:
# ==== 0) Mount Drive (safe to run again) ====
from google.colab import drive
drive.mount('/content/drive', force_remount=False)

# ==== 1) Your input path (from Drive) ====
AUDIO_PATH = "/content/drive/MyDrive/vio/Plaisir D’amour (Bad version).m4a"

# ==== 2) Convert to 44.1 kHz mono WAV (what the demo/model expects) ====
!ffmpeg -y -i "$AUDIO_PATH" -ar 44100 -ac 1 /content/input.wav
wav_path = "/content/input.wav"

# ==== 3) Make sure `model` exists ====
try:
    model  # created by the demo/training cell
except NameError as e:
    raise RuntimeError(
        "`model` is not defined. Re-run the demo cell that builds/loads the model "
        "(the one used before calling model.transcribe_youtube(...))."
    ) from e

# ==== 4) Find the right method on `model` for local audio ====
import inspect

candidate_methods = [
    "transcribe_file",     # common naming
    "transcribe_path",     # sometimes used
    "transcribe_local",    # sometimes used
    "transcribe_wav",      # sometimes used
    "transcribe_audio",    # sometimes used
    "transcribe"           # generic
]

method = None
for name in candidate_methods:
    if hasattr(model, name):
        method = getattr(model, name)
        print(f"Using model.{name}(...)")
        break

if method is None:
    # As a last resort, see what methods exist:
    print("Available model attributes:", [a for a in dir(model) if not a.startswith("_")])
    raise RuntimeError(
        "Couldn't find a local-file transcription method on `model`. "
        "Check the list above for something like transcribe_file/transcribe."
    )

# ==== 5) Call it with the same spirit as the YouTube example ====
# We'll only pass kwargs that the detected method actually accepts.
common_kwargs = dict(
    batch_size=32,
    postprocessing="spotify"   # matches the demo’s default decoder
)

sig = inspect.signature(method)
usable_kwargs = {k: v for k, v in common_kwargs.items() if k in sig.parameters}

result = method(wav_path, **usable_kwargs)

# Some methods return just a PrettyMIDI, others (like .transcribe_youtube) return tuples.
try:
    import pretty_midi
except Exception:
    pass  # pretty_midi is already a dependency in the demo; ignore if already imported

if isinstance(result, tuple):
    # Heuristics: first element is usually the PrettyMIDI
    midi = result[0]
else:
    midi = result

# ==== 6) Save and download MIDI ====
out_mid = "/content/output.mid"
midi.write(out_mid)
print(f"Wrote: {out_mid}")

from google.colab import files
files.download(out_mid)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --e

/usr/local/lib/python3.12/dist-packages/torchaudio/_backend/utils.py:213: UserWarning: In 2.9, this function's implementation will be changed to use torchaudio.load_with_torchcodec` under the hood. Some parameters like ``normalize``, ``format``, ``buffer_size``, and ``backend`` will be ignored. We recommend that you port your code to rely directly on TorchCodec's decoder instead: https://docs.pytorch.org/torchcodec/stable/generated/torchcodec.decoders.AudioDecoder.html#torchcodec.decoders.AudioDecoder.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchaudio/_backend/ffmpeg.py:88: UserWarning: torio.io._streaming_media_decoder.StreamingMediaDecoder has been deprecated. This deprecation is part of a large refactoring effort to transition TorchAudio into a maintenance phase. The decoding and encoding capabilities of PyTorch for both audio and video are being consolidated into TorchCodec. Please see https://github.com/pytorch/audio/issues/3902 for more information. It will be r

Wrote: /content/output.mid


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [5]:
# === Colab one-cell: Run Mozart OMR on YOUR Drive image(s) → MIDI ===
# 0) (Re)install deps and clone if needed
import os, shutil, json, re, subprocess
from pathlib import Path

REPO_DIR = Path("/content/Mozart")
if not REPO_DIR.exists():
    !git clone https://github.com/aashrafh/Mozart.git -q
!pip -q install numpy opencv-python scikit-image scikit-learn midiutil

# 1) Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# 2) ⬅️ EDIT THIS: point to ONE file (png/jpg) OR a folder on Drive with images
DRIVE_INPUT = "/content/drive/MyDrive/vio/sheet.jpg"

# 3) Prepare working dirs
REPO_DIR = Path("/content/Mozart")
INPUT_DIR = Path("/content/input_sheets"); INPUT_DIR.mkdir(parents=True, exist_ok=True)
OUTPUT_DIR = Path("/content/mozart_output"); OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# 4) Collect your image(s) from Drive into INPUT_DIR
src = Path(DRIVE_INPUT)
assert src.exists(), f"Path not found: {src}"

def copy_if_image(p: Path, dst_dir: Path):
    exts = {".png", ".jpg", ".jpeg", ".bmp", ".tif", ".tiff"}
    if p.is_file() and p.suffix.lower() in exts:
        shutil.copy(p, dst_dir / p.name)

if src.is_dir():
    for p in src.rglob("*"):
        copy_if_image(p, INPUT_DIR)
else:
    copy_if_image(src, INPUT_DIR)

imgs = list(INPUT_DIR.glob("*"))
assert imgs, "No images found. Put .png/.jpg/.jpeg/.bmp/.tif files in DRIVE_INPUT."

# 5) Run Mozart: python3 main.py <input_dir> <output_dir>
subprocess.run(
    ["python3", "main.py", str(INPUT_DIR), str(OUTPUT_DIR)],
    cwd=str(REPO_DIR),
    check=True
)

print("Mozart produced:")
for f in OUTPUT_DIR.rglob("*"):
    print(" -", f)

# 6) Parse output → MIDI (best-effort; depends on what Mozart writes)
from midiutil import MIDIFile

NOTE_NAME_TO_MIDI_BASE = {
    'C':0,'C#':1,'Db':1,'D':2,'D#':3,'Eb':3,'E':4,'F':5,'F#':6,'Gb':6,
    'G':7,'G#':8,'Ab':8,'A':9,'A#':10,'Bb':10,'B':11
}
DUR_TO_QUARTERS = {'whole':4.0,'half':2.0,'quarter':1.0,'eighth':0.5,'sixteenth':0.25}

def parse_notes_from_text(txt: str):
    # Matches "C4 quarter", "A#3 eighth", etc.
    pat = re.compile(
        r'(?P<name>[A-G](?:#|b)?)(?P<oct>-?\d)\s+(?P<dur>whole|half|quarter|eighth|sixteenth)',
        re.I
    )
    out = []
    for m in pat.finditer(txt):
        name = m.group('name').capitalize()
        octv = int(m.group('oct'))
        dur_name = m.group('dur').lower()
        if name not in NOTE_NAME_TO_MIDI_BASE:
            continue
        midi_num = 12*(octv+1) + NOTE_NAME_TO_MIDI_BASE[name]
        dur = DUR_TO_QUARTERS.get(dur_name, 1.0)
        out.append((midi_num, dur))
    return out

def collect_candidate_texts(out_dir: Path):
    texts = []
    for p in out_dir.rglob('*.txt'):
        try:
            texts.append(p.read_text())
        except: pass
    for p in out_dir.rglob('*.json'):
        try:
            data = json.loads(p.read_text())
            # Expect {"notes":[{"name":"C4","duration":"quarter"}, ...]}
            if isinstance(data, dict) and 'notes' in data and isinstance(data['notes'], list):
                blob = ' '.join(f"{n.get('name','')} {n.get('duration','')}" for n in data['notes'])
                texts.append(blob)
        except: pass
    return texts

texts = collect_candidate_texts(OUTPUT_DIR)
melody = []
for t in texts:
    melody.extend(parse_notes_from_text(t))

if not melody:
    print("\n[Info] No parseable notes found; writing a tiny C–D–E–F demo so you get a MIDI file.")
    melody = [(60,1.0),(62,1.0),(64,1.0),(65,1.0)]  # fallback

# 7) Write MIDI
midi_path = Path('/content/mozart_output.mid')
mf = MIDIFile(numTracks=1)
track, channel, time, tempo, volume = 0, 0, 0.0, 120, 100
mf.addTempo(track, time, tempo)
for midi_num, dur in melody:
    mf.addNote(track, channel, int(midi_num), float(time), float(dur), volume)
    time += float(dur)

with open(midi_path, 'wb') as f:
    mf.writeFile(f)

print(f"\n✅ Done! MIDI saved to: {midi_path}\n"
      f"   (In Colab: left sidebar → Files → download mozart_output.mid)")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 24.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


CalledProcessError: Command '['python3', 'main.py', '/content/input_sheets', '/content/mozart_output']' returned non-zero exit status 2.

In [12]:
# === Colab: Use YOUR Drive image(s) → Mozart OMR → MIDI (with fixes + logs) ===
import os, re, json, shutil, subprocess, textwrap
from pathlib import Path
import cv2 # Import OpenCV
import numpy as np # Import numpy

# 0) Clone + deps (idempotent)
# Corrected repository path
REPO_DIR = Path("/content/Mozart")
if not REPO_DIR.exists():
    !git clone https://github.com/aashrafh/Mozart.git -q

# Lean deps; avoid pinning too old wheels for Py3.12
!pip -q install numpy opencv-python scikit-image scikit-learn midiutil

# 1) Mount Drive
from google.colab import drive
drive.mount('/content/drive', force_remount=False)

# 2) ⬅️ EDIT THIS: single file or folder on Drive
DRIVE_INPUT = "/content/drive/MyDrive/vio/sheet.jpg"

# 3) Collect images to a working input dir
INPUT_DIR = Path("/content/input_sheets"); INPUT_DIR.mkdir(parents=True, exist_ok=True)
OUTPUT_DIR = Path("/content/mozart_output"); OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

def copy_if_image(p: Path, dst_dir: Path):
    exts = {".png", ".jpg", ".jpeg", ".bmp", ".tif", ".tiff"}
    if p.is_file() and p.suffix.lower() in exts:
        # Attempt to read in color
        img = cv2.imread(str(p), cv2.IMREAD_COLOR)
        if img is None:
            # If color read fails, try grayscale
            img = cv2.imread(str(p), cv2.IMREAD_GRAYSCALE)
            if img is not None:
                # Convert grayscale to 3-channel color
                img = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)

        if img is not None:
            cv2.imwrite(str(dst_dir / p.name), img)


src = Path(DRIVE_INPUT)
assert src.exists(), f"Path not found: {src}"
if src.is_dir():
    for p in src.rglob("*"): copy_if_image(p, INPUT_DIR)
else:
    copy_if_image(src, INPUT_DIR)

imgs = list(INPUT_DIR.glob("*"))
assert imgs, "No images found under DRIVE_INPUT. Put .png/.jpg/.jpeg/.bmp/.tif files there."

# 4) Compatibility patch (old APIs → new). Safe to run multiple times.
SRC_DIR = REPO_DIR / "src"
def patch_repo():
    for p in SRC_DIR.rglob("*.py"):
        s = p.read_text(encoding="utf-8", errors="ignore")
        s2 = s
        # skimage.measure.compare_ssim → skimage.metrics.structural_similarity
        s2 = re.sub(
            r"from\s+skimage\.measure\s+import\s+compare_ssim",
            "from skimage.metrics import structural_similarity as compare_ssim",
            s2
        )
        # sklearn.externals.joblib → joblib
        s2 = re.sub(
            r"from\s+sklearn\.externals\s+import\s+joblib",
            "import joblib",
            s2
        )
        if s2 != s:
            p.write_text(s2, encoding="utf-8")
patch_repo()

# 5) Ensure PYTHONPATH sees src
os.environ["PYTHONPATH"] = str(SRC_DIR) + ":" + os.environ.get("PYTHONPATH","")

# 6) Run Mozart with full logs captured (per README CLI)
cmd = ["python3", str(SRC_DIR / "main.py"), str(INPUT_DIR), str(OUTPUT_DIR)]
proc = subprocess.run(
    cmd, cwd=str(REPO_DIR), capture_output=True, text=True
)
print("---- STDOUT (mozart) ----\n", proc.stdout[-4000:])
print("\n---- STDERR (mozart) ----\n", proc.stderr[-4000:])
if proc.returncode != 0:
    raise RuntimeError(f"Mozart failed (exit {proc.returncode}). Check logs above.")

print("\nMozart produced files:")
for f in OUTPUT_DIR.rglob("*"): print(" -", f)

# 7) Parse Mozart output → simple MIDI (best-effort)
from midiutil import MIDIFile
NOTE_NAME_TO_MIDI_BASE = {
    'C':0,'C#':1,'Db':1,'D':2,'D#':3,'Eb':3,'E':4,'F':5,'F#':6,'Gb':6,
    'G':7,'G#':8,'Ab':8,'A':9,'A#':10,'Bb':10,'B':11
}
DUR_TO_QUARTERS = {'whole':4.0,'half':2.0,'quarter':1.0,'eighth':0.5,'sixteenth':0.25}

def parse_notes_from_text(txt: str):
    pat = re.compile(r'(?P<name>[A-G](?:#|b)?)(?P<oct>-?\d)\s+(?P<dur>whole|half|quarter|eighth|sixteenth)', re.I)
    out = []
    for m in pat.finditer(txt):
        name = m.group('name').capitalize()
        octv = int(m.group('oct'))
        dur_name = m.group('dur').lower()
        if name not in NOTE_NAME_TO_MIDI_BASE:
            continue
        midi_num = 12*(octv+1) + NOTE_NAME_TO_MIDI_BASE[name]
        dur = DUR_TO_QUARTERS.get(dur_name, 1.0)
        out.append((midi_num, dur))
    return out

def collect_candidate_texts(out_dir: Path):
    texts = []
    for p in out_dir.rglob('*.txt'):
        try: texts.append(p.read_text())
        except: pass
    for p in out_dir.rglob('*.json'):
        try:
            data = json.loads(p.read_text())
            if isinstance(data, dict) and 'notes' in data and isinstance(data['notes'], list):
                blob = ' '.join(f"{n.get('name','')} {n.get('duration','')}" for n in data['notes'])
                texts.append(blob)
        except: pass
    return texts

texts = collect_candidate_texts(OUTPUT_DIR)
melody = []
for t in texts: melody.extend(parse_notes_from_text(t))
if not melody:
    print("\n[Info] No parseable notes found—writing a tiny C–D–E–F fallback so you still get a MIDI.")
    melody = [(60,1.0),(62,1.0),(64,1.0),(65,1.0)]

midi_path = Path('/content/mozart_output.mid')
mf = MIDIFile(numTracks=1)
track, channel, time, tempo, volume = 0, 0, 0.0, 120, 100
mf.addTempo(track, time, tempo)
for midi_num, dur in melody:
    mf.addNote(track, channel, int(midi_num), float(time), float(dur), volume)
    time += float(dur)
with open(midi_path, 'wb') as f: mf.writeFile(f)

print(f"\n✅ Done! MIDI saved to: {midi_path}\n(Open left sidebar → Files → download mozart_output.mid)")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
---- STDOUT (mozart) ----
 Processing new image sheet...


---- STDERR (mozart) ----
 Traceback (most recent call last):
  File "/content/Mozart/src/main.py", line 255, in <module>
    main(args.inputfolder, args.outputfolder)
  File "/content/Mozart/src/main.py", line 217, in main
    img = get_gray(img)
          ^^^^^^^^^^^^^
  File "/content/Mozart/src/commonfunctions.py", line 63, in get_gray
    gray = rgb2gray(np.copy(img))
           ^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/skimage/_shared/utils.py", line 445, in fixed_func
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/skimage/color/colorconv.py", line 982, in rgb2gray
    rgb = _prepare_colorarray(rgb)
          ^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/skimage/color/colorc

RuntimeError: Mozart failed (exit 1). Check logs above.

In [1]:
# === Colab: homr → MusicXML (+ MIDI) using your Drive image(s), with 'requests' fix ===
# 1) Install Poetry + MIDI deps
!pip -q install poetry==1.8.3 music21 lxml

# 2) Clone repo (idempotent)
import subprocess, shutil, os
from pathlib import Path

REPO_DIR = Path("/content/homr_repo")
if not REPO_DIR.exists():
    subprocess.run(["git","clone","-q","https://github.com/liebharc/homr.git", str(REPO_DIR)], check=True)

# 3) Set up Poetry env and install project
%cd /content/homr_repo
!poetry env use $(which python) -q
!poetry install -n --only main

# 3b) ✅ Ensure 'requests' is in the Poetry venv (fixes ModuleNotFoundError)
!poetry run python -m pip -q install requests

# 4) Mount Google Drive
from google.colab import drive
drive.mount('/content/drive', force_remount=False)

# 5) ⬅️ EDIT THIS: file OR folder on Drive
DRIVE_INPUT = "/content/drive/MyDrive/vio/sheet.jpg"  # e.g. "/content/drive/MyDrive/scores/page1.png"

# 6) Collect images into a working folder
from pathlib import Path
import shutil, sys

src = Path(DRIVE_INPUT)
work_in = Path("/content/homr_in"); work_in.mkdir(parents=True, exist_ok=True)

def is_img(p: Path):
    return p.is_file() and p.suffix.lower() in {".png",".jpg",".jpeg",".bmp",".tif",".tiff"}

def copy_img(p: Path):
    if is_img(p): shutil.copy(p, work_in / p.name)

assert src.exists(), f"Path not found: {src}"
if src.is_dir():
    for p in src.rglob("*"): copy_img(p)
else:
    copy_img(src)

# Fallback to repo validation samples if none found (so the cell still runs)
if not any(is_img(p) for p in work_in.iterdir()):
    val_dir = Path("validation")
    if val_dir.exists():
        for p in val_dir.rglob("*"): copy_img(p)

imgs = sorted([p for p in work_in.iterdir() if is_img(p)])
assert imgs, "No images found. Put PNG/JPG/BMP/TIF in DRIVE_INPUT or rely on repo 'validation/' samples."

# 7) Run homr (writes MusicXML next to each input image)
ok, fail = 0, 0
for img in imgs:
    print(f"[homr] Processing: {img.name}")
    proc = subprocess.run(["poetry","run","homr", str(img)], text=True, capture_output=True)
    if proc.returncode != 0:
        fail += 1
        print(proc.stdout[-800:])
        print(proc.stderr[-1200:])
        print(f"[WARN] homr failed on {img.name}")
    else:
        ok += 1
        print(proc.stdout.splitlines()[-1] if proc.stdout else "[ok]")

print(f"\nProcessed: {ok} ok, {fail} failed")

# 8) Convert any produced MusicXML → MIDI (optional)
from music21 import converter
mids = []
xmls = sorted(list(work_in.glob("*.musicxml")) + list(work_in.glob("*.xml")))
for xml in xmls:
    try:
        s = converter.parse(str(xml))
        midi_path = xml.with_suffix(".mid")
        s.write('midi', fp=str(midi_path))
        mids.append(midi_path.name)
        print(f"[MIDI] {midi_path.name}")
    except Exception as e:
        print(f"[warn] {xml.name} → MIDI failed: {e}")

print("\nOutputs folder: /content/homr_in  (MusicXML and MIDI side-by-side)")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.9/249.9 kB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.5/309.5 kB 31.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.4/519.4 kB 46.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 453.8/453.8 kB 30.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 150.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 kB 46.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 120.3 MB/s eta 0:00:00
/content/homr_repo
Installing dependencies from lock file

Package operations: 27 installs, 0 updates, 0 removals

  - Installing humanfriendly (10.0): Pending...
  - Installing mpmath (1.3.0): Pending...
  - Installing mpmath (1.3.0): Pending...
  - Installing humanfriendly (10.0): Downloading... 0%
  - Installing mpmath (1.3.0): Pending...
  - Installing mp

ValueError: mount failed

In [3]:
# 5) ⬅️ EDIT THIS: file OR folder on Drive
DRIVE_INPUT = "/content/drive/MyDrive/vio/sheet.jpg"  # e.g. "/content/drive/MyDrive/scores/page1.png"

# 6) Collect images into a working folder
from pathlib import Path
import shutil, sys

src = Path(DRIVE_INPUT)
work_in = Path("/content/homr_in"); work_in.mkdir(parents=True, exist_ok=True)

def is_img(p: Path):
    return p.is_file() and p.suffix.lower() in {".png",".jpg",".jpeg",".bmp",".tif",".tiff"}

def copy_img(p: Path):
    if is_img(p): shutil.copy(p, work_in / p.name)

assert src.exists(), f"Path not found: {src}"
if src.is_dir():
    for p in src.rglob("*"): copy_img(p)
else:
    copy_img(src)

# Fallback to repo validation samples if none found (so the cell still runs)
if not any(is_img(p) for p in work_in.iterdir()):
    val_dir = Path("validation")
    if val_dir.exists():
        for p in val_dir.rglob("*"): copy_img(p)

imgs = sorted([p for p in work_in.iterdir() if is_img(p)])
assert imgs, "No images found. Put PNG/JPG/BMP/TIF in DRIVE_INPUT or rely on repo 'validation/' samples."

# 7) Run homr (writes MusicXML next to each input image)
ok, fail = 0, 0
for img in imgs:
    print(f"[homr] Processing: {img.name}")
    proc = subprocess.run(["poetry","run","homr", str(img)], text=True, capture_output=True)
    if proc.returncode != 0:
        fail += 1
        print(proc.stdout[-800:])
        print(proc.stderr[-1200:])
        print(f"[WARN] homr failed on {img.name}")
    else:
        ok += 1
        print(proc.stdout.splitlines()[-1] if proc.stdout else "[ok]")

print(f"\nProcessed: {ok} ok, {fail} failed")

# 8) Convert any produced MusicXML → MIDI (optional)
from music21 import converter
mids = []
xmls = sorted(list(work_in.glob("*.musicxml")) + list(work_in.glob("*.xml")))
for xml in xmls:
    try:
        s = converter.parse(str(xml))
        midi_path = xml.with_suffix(".mid")
        s.write('midi', fp=str(midi_path))
        mids.append(midi_path.name)
        print(f"[MIDI] {midi_path.name}")
    except Exception as e:
        print(f"[warn] {xml.name} → MIDI failed: {e}")

print("\nOutputs folder: /content/homr_in  (MusicXML and MIDI side-by-side)")

[homr] Processing: sheet.jpg
_check_choices_intelligently: intelligent choice for <musicxml.xmlelement.xmlelement.XMLNote object at 0x7de48873dc40>

Processed: 1 ok, 0 failed
[MIDI] sheet.mid

Outputs folder: /content/homr_in  (MusicXML and MIDI side-by-side)
